<a href="https://colab.research.google.com/github/cryptopatience/Main-code/blob/main/%EA%B8%88%EB%A6%AC_%EC%8A%A4%ED%94%84%EB%A0%88%EB%93%9C_%EA%B8%B0%EB%B0%98_%ED%88%AC%EC%9E%90_%EC%A0%84%EB%9E%B5_%EB%B6%84%EC%84%9D%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================
# 📊 금리 스프레드 기반 투자 전략 분석기 (개선 버전)
# FRED API 실시간 데이터 연동 + 결측치 처리 + 동적 기간 설정
# ============================================================================

# 필요한 라이브러리 설치
!pip install fredapi plotly pandas -q

# 라이브러리 임포트
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from fredapi import Fred
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# FRED API 설정
FRED_API_KEY = 'f1137018f7bb4b4150a5c84e09fc7fc2'
fred = Fred(api_key=FRED_API_KEY)

print("✅ 라이브러리 로딩 완료\n")

# ============================================================================
# 📅 동적 기간 설정
# ============================================================================

print("="*80)
print("📅 분석 기간 선택")
print("="*80)
print("1. 최근 60일 (기본)")
print("2. 최근 1년")
print("3. 최근 2년")
print("4. 최근 5년")
print("5. 2008년 금융위기 이후 (2007-01-01부터)")
print("6. 2000년 이후 (닷컴 버블 포함)")
print("7. 사용자 정의 기간")

period_choice = input("\n선택하세요 (1-7, 엔터=1): ").strip() or '1'

if period_choice == '1':
    analysis_days = 60
    period_name = "최근 60일"
    start_date = (datetime.now() - timedelta(days=analysis_days)).strftime('%Y-%m-%d')
elif period_choice == '2':
    analysis_days = 365
    period_name = "최근 1년"
    start_date = (datetime.now() - timedelta(days=analysis_days)).strftime('%Y-%m-%d')
elif period_choice == '3':
    analysis_days = 730
    period_name = "최근 2년"
    start_date = (datetime.now() - timedelta(days=analysis_days)).strftime('%Y-%m-%d')
elif period_choice == '4':
    analysis_days = 1825
    period_name = "최근 5년"
    start_date = (datetime.now() - timedelta(days=analysis_days)).strftime('%Y-%m-%d')
elif period_choice == '5':
    start_date = '2007-01-01'
    period_name = "2008년 금융위기 이후"
    analysis_days = (datetime.now() - datetime(2007, 1, 1)).days
elif period_choice == '6':
    start_date = '2000-01-01'
    period_name = "2000년 이후 (닷컴 버블 포함)"
    analysis_days = (datetime.now() - datetime(2000, 1, 1)).days
elif period_choice == '7':
    start_date = input("시작 날짜 입력 (YYYY-MM-DD): ").strip()
    period_name = f"{start_date}부터"
    analysis_days = (datetime.now() - datetime.strptime(start_date, '%Y-%m-%d')).days
else:
    analysis_days = 60
    period_name = "최근 60일"
    start_date = (datetime.now() - timedelta(days=analysis_days)).strftime('%Y-%m-%d')

print(f"\n✅ 선택된 기간: {period_name} ({start_date}부터)")
print(f"📊 예상 데이터 포인트: 약 {analysis_days}개")
print("="*80 + "\n")

# ============================================================================
# 데이터 로딩 함수 (결측치 처리 포함)
# ============================================================================

def fetch_series_with_ffill(series_id, start_date, end_date, name=""):
    """
    FRED API에서 데이터를 가져오고 forward fill로 결측치 처리
    """
    try:
        data = fred.get_series(series_id, start_date, end_date)
        if len(data) > 0:
            # forward fill로 결측치 채우기
            data = data.fillna(method='ffill')

            # 유효 데이터 개수
            valid_count = data.notna().sum()
            latest_date = data.dropna().index[-1].strftime('%Y-%m-%d') if len(data.dropna()) > 0 else 'N/A'

            print(f"  ✅ {name or series_id}: {len(data)}개 (유효: {valid_count}개, 최신: {latest_date})")
            return data
        else:
            print(f"  ⚠️ {name or series_id}: 데이터 없음")
            return pd.Series()
    except Exception as e:
        print(f"  ❌ {name or series_id} 수집 실패: {e}")
        return pd.Series()

def get_latest_rates(start_date_str=None, days=None):
    """
    FRED에서 금리 데이터 가져오기 (결측치 처리 포함)

    Parameters:
    - start_date_str: 시작 날짜 문자열 (YYYY-MM-DD)
    - days: 조회할 과거 일수 (start_date_str이 없을 때 사용)

    Returns:
    - DataFrame with DGS10, DGS2, EFFR
    """
    end_date = datetime.now()

    if start_date_str:
        start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    elif days:
        start_date = end_date - timedelta(days=days)
    else:
        start_date = end_date - timedelta(days=60)

    print(f"🔄 FRED 데이터 로딩 중...")
    print(f"📅 기간: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}")
    print("="*80)

    # 개별 시리즈 수집
    dgs10 = fetch_series_with_ffill('DGS10', start_date, end_date, "10년물 국채")
    dgs2 = fetch_series_with_ffill('DGS2', start_date, end_date, "2년물 국채")
    effr = fetch_series_with_ffill('EFFR', start_date, end_date, "연방기금금리")

    print("="*80)

    # 데이터프레임 생성
    df = pd.DataFrame({
        'DGS10': dgs10,
        'DGS2': dgs2,
        'EFFR': effr
    })

    # 전체 결측치 한번 더 forward fill (안전장치)
    df = df.fillna(method='ffill')

    # 결측치 제거 (모든 컬럼이 유효한 행만 유지)
    df = df.dropna()

    print(f"\n✅ 기본 DataFrame 생성 완료!")
    print(f"   - 기간: {df.index[0].strftime('%Y-%m-%d')} ~ {df.index[-1].strftime('%Y-%m-%d')}")
    print(f"   - 총 데이터 포인트: {len(df)}개")

    # 스프레드 계산
    df['Yield_Curve'] = df['DGS10'] - df['DGS2']  # 수익률 곡선
    df['Policy_Spread'] = df['DGS2'] - df['EFFR']  # 정책 기대

    # 데이터 품질 체크
    print("\n[데이터 품질 체크]")
    for col in ['DGS10', 'DGS2', 'EFFR', 'Yield_Curve', 'Policy_Spread']:
        if col in df.columns:
            valid = df[col].notna().sum()
            missing = df[col].isna().sum()
            pct_valid = (valid / len(df)) * 100 if len(df) > 0 else 0

            if missing > 0:
                print(f"  ⚠️ {col}: {valid}개 유효 ({pct_valid:.1f}%), {missing}개 결측")
            else:
                print(f"  ✅ {col}: {valid}개 유효 (100%), 결측 없음")

    print("="*80 + "\n")

    return df

# ============================================================================
# 시나리오 판별 함수
# ============================================================================

def determine_scenario(yield_curve, policy_spread):
    """
    금리 스프레드 기반 경제 시나리오 판별

    Parameters:
    - yield_curve: 10Y - 2Y (음수면 역전)
    - policy_spread: 2Y - FFR (음수면 완화 기대)

    Returns:
    - scenario_number (1~4)
    """
    inverted = yield_curve < 0
    easing_expected = policy_spread < 0

    if inverted and not easing_expected:
        return 1  # 스태그플레이션
    elif inverted and easing_expected:
        return 2  # 침체 경고
    elif not inverted and not easing_expected:
        return 3  # 건강한 성장
    else:
        return 4  # 정책 전환점

# 시나리오별 상세 정보
SCENARIOS = {
    1: {
        'title': '🟡 시나리오 1: 스태그플레이션 우려',
        'meaning': '수익률 곡선 역전 + 시장은 여전히 긴축 기대 → 인플레이션 지속 + 성장 둔화',
        'risk': '⚠️ 고위험',
        'color': '#f57f17',
        'assets': {
            '주식 (성장주)': '⚠️ 축소 (20-30%)',
            '주식 (가치주)': '✅ 유지 (30-40%)',
            '기술주': '🔴 대폭 축소 (10-15%)',
            '비트코인': '🔴 최소화 (0-5%)',
            '부동산/리츠': '⚠️ 선별적 (10-15%)',
            '채권': '⚠️ 단기채 위주 (20-30%)',
            '원자재/금': '✅ 확대 (15-20%)',
            '현금': '✅ 확보 (10-20%)'
        }
    },
    2: {
        'title': '🚨 시나리오 2: 침체 경고',
        'meaning': '수익률 곡선 역전 + 완화 기대 → 경기 침체 임박!',
        'risk': '⚠️⚠️ 최고위험',
        'color': '#c62828',
        'assets': {
            '주식 (성장주)': '🚫 매도 (0-10%)',
            '주식 (가치주)': '⚠️ 최소화 (10-20%)',
            '기술주': '🚫 전량 청산 (0%)',
            '비트코인': '🚫 매도 (0%)',
            '부동산/리츠': '🔴 청산 (0-5%)',
            '채권': '✅ 장기채 집중 (40-50%)',
            '원자재/금': '✅ 금 중심 (20-30%)',
            '현금': '✅ 최대 확보 (20-30%)'
        }
    },
    3: {
        'title': '✅ 시나리오 3: 건강한 성장',
        'meaning': '정상 곡선 + 긴축 기대 → 건강한 경제 성장',
        'risk': '✅ 저위험',
        'color': '#2e7d32',
        'assets': {
            '주식 (성장주)': '✅ 적극 매수 (40-50%)',
            '주식 (가치주)': '✅ 균형 유지 (20-30%)',
            '기술주': '✅ 공격 투자 (25-35%)',
            '비트코인': '⚠️ 선택적 (5-10%)',
            '부동산/리츠': '✅ 적극 투자 (10-20%)',
            '채권': '⚠️ 최소화 (5-10%)',
            '원자재/금': '➡️ 중립 (5-10%)',
            '현금': '➡️ 최소 (5-10%)'
        }
    },
    4: {
        'title': '🔄 시나리오 4: 정책 전환점 (Pivot)',
        'meaning': '정상 곡선 + 완화 기대 → 긴축 종료 신호',
        'risk': '➡️ 중간위험',
        'color': '#1565c0',
        'assets': {
            '주식 (성장주)': '⚠️ 조정 (25-35%)',
            '주식 (가치주)': '✅ 확대 (25-35%)',
            '기술주': '⚠️ 선별 투자 (20-25%)',
            '비트코인': '✅ 적극 매수 (10-15%)',
            '부동산/리츠': '✅ 적극 매수 (15-20%)',
            '채권': '✅ 장기채 확대 (20-30%)',
            '원자재/금': '➡️ 중립 (5-10%)',
            '현금': '➡️ 적정 수준 (10-15%)'
        }
    }
}

print("✅ 시나리오 분석 함수 준비 완료\n")

# ============================================================================
# 데이터 로드
# ============================================================================

df_rates = get_latest_rates(start_date_str=start_date)

if df_rates is not None and len(df_rates) > 0:
    print(f"✅ 데이터 로딩 완료: {len(df_rates)}개 데이터 포인트")
    print(f"📅 기간: {df_rates.index[0].strftime('%Y-%m-%d')} ~ {df_rates.index[-1].strftime('%Y-%m-%d')}")
    print(f"\n최근 5일 데이터:")
    print(df_rates[['DGS10', 'DGS2', 'EFFR', 'Yield_Curve', 'Policy_Spread']].tail().round(2))

# ============================================================================
# 최신 데이터 분석
# ============================================================================

if df_rates is not None and len(df_rates) > 0:
    latest = df_rates.iloc[-1]
    latest_date = df_rates.index[-1]

    us10y = latest['DGS10']
    us2y = latest['DGS2']
    ffr = latest['EFFR']
    yield_curve = latest['Yield_Curve']
    policy_spread = latest['Policy_Spread']

    scenario = determine_scenario(yield_curve, policy_spread)
    info = SCENARIOS[scenario]

    print("\n" + "="*80)
    print(f"📅 분석 기준일: {latest_date.strftime('%Y년 %m월 %d일')}")
    print(f"📊 분석 기간: {period_name}")
    print("="*80)
    print(f"\n💰 현재 금리 수준:")
    print(f"  10년물 금리: {us10y:.2f}%")
    print(f"  2년물 금리: {us2y:.2f}%")
    print(f"  연방기금금리: {ffr:.2f}%")
    print(f"\n📊 스프레드 분석:")
    print(f"  10Y-2Y 스프레드: {yield_curve:.2f}% {'(역전 🔴)' if yield_curve < 0 else '(정상 ✅)'}")
    print(f"  2Y-FFR 스프레드: {policy_spread:.2f}% {'(완화 기대)' if policy_spread < 0 else '(긴축 기대)'}")
    print("\n" + "="*80)
    print(info['title'])
    print("="*80)
    print(f"\n💡 의미: {info['meaning']}")
    print(f"⚖️ 위험 수준: {info['risk']}")
    print("\n🎯 자산별 권장 비중:")
    print("-" * 80)
    for asset, allocation in info['assets'].items():
        print(f"  {asset}: {allocation}")
    print("="*80)

# ============================================================================
# 시계열 차트 시각화
# ============================================================================

if df_rates is not None and len(df_rates) > 0:
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=(
            '📈 금리 추이 (10Y, 2Y, FFR)',
            '📊 수익률 곡선 스프레드 (10Y - 2Y)',
            '📈 정책 기대 스프레드 (2Y - FFR)'
        ),
        vertical_spacing=0.12,
        row_heights=[0.4, 0.3, 0.3]
    )

    # 1. 금리 추이
    fig.add_trace(
        go.Scatter(x=df_rates.index, y=df_rates['DGS10'],
                   name='10년물', line=dict(color='#1f77b4', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=df_rates.index, y=df_rates['DGS2'],
                   name='2년물', line=dict(color='#ff7f0e', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=df_rates.index, y=df_rates['EFFR'],
                   name='FFR', line=dict(color='#2ca02c', width=2)),
        row=1, col=1
    )

    # 2. 수익률 곡선 (역전 영역 표시)
    fig.add_trace(
        go.Scatter(x=df_rates.index, y=df_rates['Yield_Curve'],
                   name='10Y-2Y', line=dict(color='purple', width=2),
                   fill='tozeroy', fillcolor='rgba(128,0,128,0.1)'),
        row=2, col=1
    )
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

    # 3. 정책 기대 스프레드
    fig.add_trace(
        go.Scatter(x=df_rates.index, y=df_rates['Policy_Spread'],
                   name='2Y-FFR', line=dict(color='orange', width=2),
                   fill='tozeroy', fillcolor='rgba(255,165,0,0.1)'),
        row=3, col=1
    )
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=3, col=1)

    # 레이아웃 설정
    fig.update_xaxes(title_text="날짜", row=3, col=1)
    fig.update_yaxes(title_text="금리 (%)", row=1, col=1)
    fig.update_yaxes(title_text="스프레드 (%)", row=2, col=1)
    fig.update_yaxes(title_text="스프레드 (%)", row=3, col=1)

    fig.update_layout(
        height=1000,
        title_text=f"<b>금리 스프레드 분석 대시보드</b><br><sub>{period_name}</sub>",
        title_x=0.5,
        showlegend=True,
        hovermode='x unified'
    )

    fig.show()

    print("\n✅ 차트 생성 완료")

# ============================================================================
# 시나리오별 히스토리 분석
# ============================================================================

if df_rates is not None and len(df_rates) > 0:
    df_rates['Scenario'] = df_rates.apply(
        lambda row: determine_scenario(row['Yield_Curve'], row['Policy_Spread']),
        axis=1
    )

    scenario_counts = df_rates['Scenario'].value_counts().sort_index()

    print(f"\n📊 {period_name} 시나리오 분포:")
    print("="*80)
    for scenario_num in [1, 2, 3, 4]:
        count = scenario_counts.get(scenario_num, 0)
        pct = (count / len(df_rates)) * 100 if len(df_rates) > 0 else 0
        bar = '█' * int(pct / 2)
        scenario_title = SCENARIOS[scenario_num]['title'][:35]
        print(f"{scenario_title:35s} {count:4d}일 ({pct:5.1f}%) {bar}")

    print("="*80)

    # 시나리오 전환 타임라인
    scenario_colors = {1: '#f57f17', 2: '#c62828', 3: '#2e7d32', 4: '#1565c0'}

    fig = go.Figure()

    for scenario_num in [1, 2, 3, 4]:
        mask = df_rates['Scenario'] == scenario_num
        fig.add_trace(go.Scatter(
            x=df_rates[mask].index,
            y=[scenario_num] * mask.sum(),
            mode='markers',
            name=SCENARIOS[scenario_num]['title'][:20],
            marker=dict(size=10, color=scenario_colors[scenario_num])
        ))

    fig.update_layout(
        title=f"<b>시나리오 전환 타임라인</b><br><sub>{period_name}</sub>",
        xaxis_title="날짜",
        yaxis_title="시나리오",
        yaxis=dict(tickmode='linear', tick0=1, dtick=1),
        height=400,
        hovermode='x unified'
    )

    fig.show()

# ============================================================================
# 상세 통계 분석
# ============================================================================

if df_rates is not None and len(df_rates) > 0:
    print(f"\n📊 {period_name} 통계 요약:")
    print("="*80)

    print(f"\n[수익률 곡선 (10Y-2Y)]")
    print(f"  평균: {df_rates['Yield_Curve'].mean():.2f}%")
    print(f"  최대: {df_rates['Yield_Curve'].max():.2f}% ({df_rates['Yield_Curve'].idxmax().strftime('%Y-%m-%d')})")
    print(f"  최소: {df_rates['Yield_Curve'].min():.2f}% ({df_rates['Yield_Curve'].idxmin().strftime('%Y-%m-%d')})")
    print(f"  표준편차: {df_rates['Yield_Curve'].std():.2f}%")

    inversion_days = (df_rates['Yield_Curve'] < 0).sum()
    inversion_pct = (inversion_days / len(df_rates)) * 100 if len(df_rates) > 0 else 0
    print(f"  역전 일수: {inversion_days}일 ({inversion_pct:.1f}%)")

    print(f"\n[정책 스프레드 (2Y-FFR)]")
    print(f"  평균: {df_rates['Policy_Spread'].mean():.2f}%")
    print(f"  최대: {df_rates['Policy_Spread'].max():.2f}% ({df_rates['Policy_Spread'].idxmax().strftime('%Y-%m-%d')})")
    print(f"  최소: {df_rates['Policy_Spread'].min():.2f}% ({df_rates['Policy_Spread'].idxmin().strftime('%Y-%m-%d')})")
    print(f"  표준편차: {df_rates['Policy_Spread'].std():.2f}%")

    easing_days = (df_rates['Policy_Spread'] < 0).sum()
    easing_pct = (easing_days / len(df_rates)) * 100 if len(df_rates) > 0 else 0
    print(f"  완화 기대 일수: {easing_days}일 ({easing_pct:.1f}%)")

    print("\n" + "="*80)

# ============================================================================
# 2008년 금융위기 비교 분석 (장기 데이터인 경우)
# ============================================================================

if period_choice in ['5', '6']:  # 2008년 금융위기 이후 또는 2000년 이후
    print("\n" + "="*80)
    print("📊 2008년 금융위기 비교 분석")
    print("="*80)

    # 2008년 데이터 추출 (2007-2009)
    crisis_period = df_rates[(df_rates.index >= '2007-01-01') & (df_rates.index <= '2009-12-31')]

    if len(crisis_period) > 0:
        print(f"\n[2008년 금융위기 기간 (2007-2009)]")
        print(f"  수익률 곡선 평균: {crisis_period['Yield_Curve'].mean():.2f}%")
        print(f"  수익률 곡선 최소: {crisis_period['Yield_Curve'].min():.2f}%")
        print(f"  역전 일수: {(crisis_period['Yield_Curve'] < 0).sum()}일")
        print(f"  정책 스프레드 평균: {crisis_period['Policy_Spread'].mean():.2f}%")

        # 현재와 비교
        current_yc = df_rates['Yield_Curve'].iloc[-1]
        current_ps = df_rates['Policy_Spread'].iloc[-1]
        crisis_yc_avg = crisis_period['Yield_Curve'].mean()
        crisis_ps_avg = crisis_period['Policy_Spread'].mean()

        print(f"\n[현재 vs 2008년 비교]")
        print(f"  수익률 곡선: {current_yc:.2f}% (위기 평균: {crisis_yc_avg:.2f}%)")
        print(f"  정책 스프레드: {current_ps:.2f}% (위기 평균: {crisis_ps_avg:.2f}%)")

        if current_yc < crisis_yc_avg:
            print(f"  ⚠️ 현재 수익률 곡선이 2008년 위기 수준에 근접")
        else:
            print(f"  ✅ 현재 수익률 곡선은 2008년보다 양호")
    else:
        print("  ⚠️ 2008년 데이터 부족")

    print("="*80)

# ============================================================================
# 사용자 정의 분석 함수
# ============================================================================

def analyze_custom_rates(us10y, us2y, ffr):
    """
    사용자가 입력한 금리로 시나리오 분석
    """
    yield_curve = us10y - us2y
    policy_spread = us2y - ffr

    scenario = determine_scenario(yield_curve, policy_spread)
    info = SCENARIOS[scenario]

    print("="*80)
    print("📊 사용자 정의 금리 분석")
    print("="*80)
    print(f"\n💰 입력 금리:")
    print(f"  10년물 금리: {us10y:.2f}%")
    print(f"  2년물 금리: {us2y:.2f}%")
    print(f"  연방기금금리: {ffr:.2f}%")
    print(f"\n📊 스프레드:")
    print(f"  10Y-2Y 스프레드: {yield_curve:.2f}% {'(역전 🔴)' if yield_curve < 0 else '(정상 ✅)'}")
    print(f"  2Y-FFR 스프레드: {policy_spread:.2f}% {'(완화 기대)' if policy_spread < 0 else '(긴축 기대)'}")
    print("\n" + "="*80)
    print(info['title'])
    print("="*80)
    print(f"\n💡 의미: {info['meaning']}")
    print(f"⚖️ 위험: {info['risk']}")
    print("\n🎯 자산별 권장 비중:")
    print("-" * 80)
    for asset, allocation in info['assets'].items():
        print(f"  {asset}: {allocation}")
    print("="*80)

# 예시 분석
print("\n\n" + "="*80)
print("💡 예시 분석")
print("="*80)

print("\n예시 1: 정상 성장 시나리오")
print("-" * 80)
analyze_custom_rates(us10y=4.5, us2y=4.2, ffr=5.0)

print("\n\n예시 2: 침체 우려 시나리오")
print("-" * 80)
analyze_custom_rates(us10y=3.8, us2y=4.3, ffr=5.0)

# ============================================================================
# 데이터 저장
# ============================================================================

if df_rates is not None:
    filename = f"rate_analysis_{period_name.replace(' ', '_')}_{datetime.now().strftime('%Y%m%d')}.csv"
    df_rates.to_csv(filename)
    print(f"\n✅ 데이터 저장 완료: {filename}")

    # 데이터 다운로드 (코랩에서만 실행)
    try:
        from google.colab import files
        files.download(filename)
        print(f"📥 파일 다운로드 시작")
    except:
        print(f"💾 로컬 파일로 저장됨: {filename}")

# ============================================================================
# 해석 가이드
# ============================================================================

print("\n\n" + "="*80)
print("📖 금리 스프레드 해석 가이드")
print("="*80)

print("\n[수익률 곡선 (10Y-2Y)]")
print("✅ 양수 (정상): 경제가 건강하게 성장 중")
print("   → 장기 금리 > 단기 금리")
print("   → 은행들이 대출을 활발히 진행")
print()
print("🔴 음수 (역전): 경기 침체 전조 신호 (12-18개월 선행)")
print("   → 장기 금리 < 단기 금리")
print("   → 시장이 미래 경기 침체 예상")
print("   → 역사적으로 모든 역전 후 침체 발생")

print("\n[정책 스프레드 (2Y-FFR)]")
print("✅ 양수: 시장이 금리 인상 또는 긴축 유지 예상")
print("   → 인플레이션 통제 필요")
print("   → 성장 지속 가능")
print()
print("🔴 음수: 시장이 금리 인하(완화) 예상")
print("   → 경기 둔화 우려")
print("   → 연준의 정책 전환 임박")

print("\n[시나리오별 투자 전략]")
print("1️⃣ 스태그플레이션: 방어적 → 현금/금/단기채 중심")
print("2️⃣ 침체 경고: 초방어적 → 장기채/금/현금 최대")
print("3️⃣ 건강한 성장: 공격적 → 주식(성장주)/부동산 집중")
print("4️⃣ 정책 전환점: 균형잡힌 → 비트코인/부동산/가치주")

print("="*80)

print("\n\n" + "="*80)
print("🎉 모든 분석 완료!")
print("="*80)
print(f"📊 분석 기간: {period_name}")
print(f"📅 최신 데이터: {df_rates.index[-1].strftime('%Y-%m-%d')}")
print(f"🎯 현재 시나리오: {SCENARIOS[scenario]['title']}")
print("="*80)

✅ 라이브러리 로딩 완료

📅 분석 기간 선택
1. 최근 60일 (기본)
2. 최근 1년
3. 최근 2년
4. 최근 5년
5. 2008년 금융위기 이후 (2007-01-01부터)
6. 2000년 이후 (닷컴 버블 포함)
7. 사용자 정의 기간


 **상황에 따라 다르지만**, 일반적으로는 **2년물이 연방기금금리보다 높은 것이 정상이고 건강한 신호**입니다.

## 📊 2년물 vs 연방기금금리 관계 분석

### ✅ 2년물 > 연방기금금리 (정상/긴축 기대)

**의미**: 시장이 연준의 추가 금리 인상을 예상

**왜 정상적인가?**
- 2년물 국채는 향후 2년간의 금리 평균을 반영
- 연준이 금리를 더 올릴 것으로 예상되면 2년물이 현재 FFR보다 높아짐
- 경제가 건강하게 성장 중이라는 신호

**장점** 👍
1. **경제 성장 지속**: GDP 성장, 고용 증가
2. **기업 실적 호조**: 매출 증가, 투자 확대
3. **주식시장 강세**: 특히 경기 순환주, 금융주
4. **인플레이션 통제 가능**: 연준이 주도권 유지

**단점** 👎
1. **대출 비용 상승**: 기업/개인 자금 조달 부담
2. **채권 가격 하락**: 금리 상승기에는 채권 손실
3. **성장주 압박**: 높은 할인율로 밸류에이션 부담

**예시**:
- 2017-2018년: 2Y > FFR, S&P 500 +20% 상승
- 현재가 4.3% (2Y), 연방기금금리 4.5% → 스프레드 -0.2%

---

### ⚠️ 2년물 < 연방기금금리 (완화 기대)

**의미**: 시장이 연준의 금리 인하를 예상

**왜 발생하나?**
- 경기 둔화 우려로 연준이 금리를 내릴 것으로 예상
- 향후 2년간 평균 금리가 현재보다 낮을 것으로 전망
- 인플레이션 진정 또는 경기 침체 우려

**언제 좋은가? (정책 전환점 Pivot)** ✅
1. **경기가 여전히 양호한데** 연준이 긴축을 멈출 때
2. **소프트랜딩** 성공 가능성
3. **금리 인하 수혜주** 상승: 리츠, 성장주, 비트코인
4. **장기 채권 랠리**: 금리 하락으로 가격 상승

**예시 - 성공 사례**:
- **2019년**: 2Y < FFR → 연준 3차례 예방적 금리 인하 → 주식 강세
- **1995년**: 소프트랜딩 성공 → 이후 5년간 대세 상승장

**언제 나쁜가? (침체 경고)** 🚨
1. **수익률 곡선도 역전**되어 있을 때 (10Y < 2Y)
2. **경기 지표 악화**: 실업률 상승, 소비 둔화
3. **신용 경색**: 은행 대출 축소, 부실 증가
4. **자산 가격 폭락**: 주식, 부동산 동반 하락

**예시 - 실패 사례**:
- **2007년**: 2Y < FFR + 수익률 곡선 역전 → 2008년 금융위기
- **2001년**: 2Y < FFR → 닷컴 버블 붕괴 → 경기 침체

---

## 🎯 핵심 정리

### 상황별 판단 기준

| 상황 | 2Y vs FFR | 수익률 곡선 | 의미 | 투자 전략 |
|------|-----------|------------|------|-----------|
| **최고** 😊 | 2Y > FFR | 정상 (10Y > 2Y) | 건강한 성장 | 공격적 투자 |
| **좋음** 🙂 | 2Y < FFR | 정상 (10Y > 2Y) | 정책 전환점 | 금리 민감주 매수 |
| **주의** 😐 | 2Y > FFR | 역전 (10Y < 2Y) | 스태그플레이션 | 방어적 전환 |
| **위험** 😰 | 2Y < FFR | 역전 (10Y < 2Y) | 침체 임박 | 현금/채권/금 |

### 실전 투자 가이드

**2년물 > 연방기금금리일 때** (스프레드 +0.5% 이상)
```
✅ 주식 비중 확대 (60-80%)
✅ 성장주 적극 매수
✅ 경기 순환주 선호
⚠️ 채권 비중 축소
⚠️ 금리 인상 리스크 관리
```

**2년물 ≈ 연방기금금리** (스프레드 ±0.3% 이내)
```
➡️ 중립 포지션 유지
➡️ 연준 발언 주시
➡️ 변동성 대비
```

**2년물 < 연방기금금리일 때** (스프레드 -0.5% 이하)
```
🔍 수익률 곡선 확인 필수!

정상 곡선이면:
✅ 리츠/부동산 매수
✅ 장기 채권 매수
✅ 비트코인 적극 매수
✅ 성장주 재진입

역전 곡선이면:
🚨 방어 포지션 전환
🚨 현금 비중 확대 (20-30%)
🚨 장기 국채 집중 (40-50%)
🚨 위험자산 최소화
```

---

## 💡 현재 시장 체크리스트

**지금 확인해야 할 것들:**

1. **2Y-FFR 스프레드** = ?
   - 양수(+): 긴축 기대 → 경제 건강
   - 음수(-): 완화 기대 → 전환점 or 위기?

2. **10Y-2Y 스프레드** = ?
   - 양수(+): 정상 → 안전
   - 음수(-): 역전 → 경고

3. **두 개를 조합**해서 4대 시나리오 판별

**결론**:
- **절대적으로 좋고 나쁨은 없습니다**
- **다른 지표와 함께 봐야** 정확한 판단 가능
- **일반적으로는 2Y > FFR이 건강한 신호**지만
- **2Y < FFR도 Pivot 시점에는 큰 기회**가 될 수 있습니다!

지금 코드를 돌려서 현재 어떤 상황인지 확인해보시는 게 가장 정확합니다! 🎯

# 📊 금리 스프레드 4대 시나리오 상세 설명

## 🟡 시나리오 1: 스태그플레이션 우려

### 📌 발생 조건
- **수익률 곡선**: 역전 (10년물 < 2년물)
- **정책 기대**: 긴축 지속 (2년물 > 연방기금금리)

### 💡 경제적 의미
시장은 장기적으로 경기 침체를 예상하고 있지만(수익률 곡선 역전), 연준은 여전히 인플레이션과 싸우기 위해 금리를 더 올릴 것으로 예상되는 상황입니다. 이는 1970년대 오일쇼크 때와 유사한 **스태그플레이션**(경기 침체 + 인플레이션 동반)의 위험을 시사합니다.

### 🔍 실제 사례
- **2022년 중반**: 러시아-우크라이나 전쟁으로 에너지/식량 가격 급등, 연준은 40년 만의 공격적 금리 인상 단행
- 수익률 곡선은 역전되었지만 시장은 연준이 인플레이션 잡기 위해 계속 금리를 올릴 것으로 예상

### ⚠️ 위험 요인
- **기업**: 높은 금리로 자금 조달 비용 급증 → 수익성 악화
- **소비자**: 물가 상승 + 고금리로 대출/신용카드 부담 증가
- **자산시장**: 성장주/기술주는 미래 현금흐름 할인율 상승으로 밸류에이션 타격

### 💼 투자 전략
**방어적 포트폴리오 구성**

1. **주식 (50-70% → 축소)**
   - 성장주 20-30%로 축소: 고금리 환경에서 미래 이익의 현재가치 하락
   - 가치주 30-40% 유지: 방어주, 배당주 중심 (필수소비재, 에너지, 헬스케어)
   - 기술주 10-15%로 대폭 축소: 높은 밸류에이션 부담

2. **비트코인 0-5% 최소화**
   - 고금리 + 경기 둔화는 위험자산에 최악
   - 유동성 축소로 가격 하락 압력

3. **부동산/리츠 10-15% 선별적**
   - 고금리로 대출 부담 증가
   - 필수 시설 리츠(물류센터, 데이터센터)만 선별

4. **채권 20-30% 단기채 위주**
   - 금리 상승기에는 장기채 손실 위험
   - 단기 국채/회사채로 유동성 확보

5. **원자재/금 15-20% 확대**
   - 인플레이션 헤지: 금, 은, 에너지
   - 실물자산의 가치 보존 효과

6. **현금 10-20% 확보**
   - 고금리 환경에서 머니마켓펀드 수익률 매력적
   - 급락 시 매수 기회 대기

---

## 🚨 시나리오 2: 침체 경고 (최고 위험)

### 📌 발생 조건
- **수익률 곡선**: 역전 (10년물 < 2년물)
- **정책 기대**: 완화 전환 (2년물 < 연방기금금리)

### 💡 경제적 의미
수익률 곡선이 역전되었고, 시장은 연준이 곧 금리를 내릴 것으로 예상합니다. 역사적으로 이 조합은 **6~18개월 내 경기 침체 발생**의 강력한 신호입니다. 연준이 금리를 내리는 이유는 보통 경제가 이미 악화되었거나 급격히 나빠지고 있기 때문입니다.

### 🔍 실제 사례
- **2007년 중반**: 서브프라임 모기지 위기 징후, 수익률 곡선 역전 + 완화 기대
  - 2007년 9월 연준 금리 인하 시작
  - 2008년 9월 리먼 브라더스 파산 → 금융위기
  
- **2000년 초**: 닷컴 버블 붕괴 직전
  - 수익률 곡선 역전
  - 2001년 1월부터 연준 긴급 금리 인하
  - 나스닥 78% 폭락

### ⚠️ 위험 요인
- **실업률 급등**: 기업들의 대규모 구조조정
- **신용경색**: 은행 대출 축소, 회사채 시장 경색
- **자산 가격 폭락**: 주식, 부동산, 암호화폐 동반 하락
- **디플레이션 위험**: 수요 급감으로 물가 하락 → 기업 수익성 악화

### 💼 투자 전략
**극단적 방어 포지션 (생존 모드)**

1. **주식 10-30%로 최소화**
   - 성장주 0-10% 매도: 침체 시 가장 큰 타격
   - 가치주 10-20% 최소화: 방어주(헬스케어, 필수소비재, 유틸리티)만 극소량
   - 기술주 전량 청산: 수익성 없는 기업들 도산 위험

2. **비트코인 전량 매도**
   - 침체기 유동성 위기 시 가장 먼저 매도되는 자산
   - 2022년 FTX 사태처럼 시스템 리스크 발생 가능

3. **부동산/리츠 0-5% 청산**
   - 금리 부담 + 경기 침체로 이중 타격
   - 공실률 상승, 임대료 하락

4. **채권 40-50% 장기채 집중 (핵심 전략)**
   - 연준의 금리 인하로 장기채 가격 급등
   - 안전자산 선호로 국채 수요 폭증
   - 2008년 금융위기 때 30년물 국채 수익률 40% 기록

5. **원자재/금 20-30% 금 중심**
   - 금: 공포 지수 상승 시 안전자산
   - 에너지: 수요 감소로 가격 하락 예상

6. **현금 20-30% 최대 확보**
   - 급락장에서 저가 매수 기회 대기
   - 유동성이 생명줄

### 🎯 행동 지침
- ⚠️ 즉시 포트폴리오 재조정 필요
- 📉 자산 가격 30-50% 하락 각오
- 💰 생활비 6-12개월치 현금 확보
- 🚫 레버리지/신용 거래 절대 금지

---

## ✅ 시나리오 3: 건강한 성장 (최적 환경)

### 📌 발생 조건
- **수익률 곡선**: 정상 (10년물 > 2년물)
- **정책 기대**: 긴축 지속 (2년물 > 연방기금금리)

### 💡 경제적 의미
경제가 건강하게 성장하고 있어 장기 금리가 단기 금리보다 높고(정상적인 수익률 곡선), 연준은 경기 과열과 인플레이션을 막기 위해 금리를 더 올릴 것으로 예상되는 상황입니다. 이는 **골디락스 경제**(너무 뜨겁지도, 차갑지도 않은)의 이상적 환경입니다.

### 🔍 실제 사례
- **2017-2018년**: 트럼프 감세 효과 + 기업 실적 호조
  - GDP 성장률 2.5-3%
  - 실업률 50년 만에 최저
  - S&P 500 연간 20% 상승
  
- **2004-2006년**: 주택 시장 호황기
  - 연준의 점진적 금리 인상
  - 기업 투자 확대, 소비 견조

### 🌟 긍정 요인
- **기업 실적 호조**: 매출 증가 + 마진 개선
- **고용 시장 강세**: 임금 상승, 소비 증가
- **투자 확대**: 설비 투자, R&D 증가
- **신용 건전성 양호**: 부실 채권 비율 낮음

### 💼 투자 전략
**공격적 성장 포트폴리오**

1. **주식 60-80% 적극 투자**
   - 성장주 40-50% 적극 매수: 기술 혁신, 시장 점유율 확대 기업
   - 가치주 20-30% 균형 유지: 금융, 산업재, 소비재
   - 기술주 25-35% 공격 투자: AI, 클라우드, 반도체, 소프트웨어

2. **비트코인 5-10% 선택적**
   - 리스크 감내 가능 투자자만
   - 성장 환경에서 위험자산 선호 증가

3. **부동산/리츠 10-20% 적극 투자**
   - 상업용 부동산: 임대 수요 증가
   - 데이터센터 리츠: 클라우드 수요 폭발적 증가
   - 물류센터: 전자상거래 성장

4. **채권 5-10% 최소화**
   - 금리 상승기에는 채권 수익률 낮음
   - 주식 대비 기회비용 큼

5. **원자재 5-10% 중립**
   - 산업재 중심: 구리, 알루미늄 등
   - 인프라 투자 수혜

6. **현금 5-10% 최소**
   - 고수익 자산에 최대한 배분
   - 최소한의 유동성만 확보

### 🎯 세부 전략
- **섹터 로테이션**: 경기 순환 섹터(금융, 산업재, 소비재) 비중 확대
- **중소형주**: 대형주 대비 높은 성장 잠재력
- **해외 주식**: 신흥국 포함 글로벌 분산

### 📈 예상 수익률
- 연간 15-25% 수익 가능
- 변동성은 있지만 장기적으로 상승

---

## 🔄 시나리오 4: 정책 전환점 (Pivot)

### 📌 발생 조건
- **수익률 곡선**: 정상 (10년물 > 2년물)
- **정책 기대**: 완화 전환 (2년물 < 연방기금금리)

### 💡 경제적 의미
경제는 아직 정상이지만(수익률 곡선 정상), 시장은 연준이 긴축을 멈추거나 금리를 내릴 것으로 예상합니다. 이는 **정책 전환점(Pivot)**으로, 경기가 둔화 조짐을 보이거나 연준이 인플레이션을 충분히 잡았다고 판단하는 시점입니다.

### 🔍 실제 사례
- **2019년 중반**: 무역 전쟁 우려로 성장 둔화
  - 2018년 긴축 종료 선언
  - 2019년 7월부터 예방적 금리 인하
  - 코로나 이전까지 주식시장 랠리
  
- **1995년**: 연준의 소프트 랜딩 성공 사례
  - 1994-95년 긴축 후 완화로 전환
  - 닷컴 버블까지 5년간 강세장

### 🎯 시장 특징
- **금리 민감 자산 강세**: 부동산, 리츠, 성장주
- **밸류에이션 재평가**: 할인율 하락으로 주가 상승
- **변동성 증가**: 정책 불확실성으로 단기 등락
- **채권 랠리**: 금리 하락 기대로 장기채 수요 증가

### 💼 투자 전략
**균형 잡힌 포트폴리오 + 금리 민감주 비중 확대**

1. **주식 50-70% 조정**
   - 성장주 25-35% 조정: 변동성 대비하되 장기 성장 기업 유지
   - 가치주 25-35% 확대: 안정적 배당주, 방어주
   - 기술주 20-25% 선별: 대형 우량주(빅테크) 위주
   - 추천 섹터: 유틸리티, 통신, 리츠

2. **비트코인 10-15% 적극 매수 (핵심 전략)**
   - 금리 인하는 암호화폐에 가장 유리한 환경
   - 2019년, 2020년 금리 인하기 비트코인 급등 사례
   - 유동성 증가로 위험자산 선호도 상승

3. **부동산/리츠 15-20% 적극 매수**
   - 금리 하락의 최대 수혜 자산
   - 부동산 가치 재평가
   - 리츠 배당 수익률 매력 증가

4. **채권 20-30% 장기채 확대**
   - 금리 하락 시 장기채 가격 상승
   - 듀레이션 길수록 수익률 높음
   - 10년물, 30년물 국채 비중 확대

5. **원자재 5-10% 중립**
   - 금: 실질금리 하락 시 수혜
   - 기타 원자재: 경기 둔화로 중립

6. **현금 10-15% 적정 수준**
   - 변동성 대비 일부 현금 확보
   - 급락 시 매수 기회 포착

### 🎯 투자 타이밍 전략

**초기 (Pivot 신호 감지)**
- 금리 민감 자산(리츠, 장기채) 먼저 매수
- 비트코인 선제적 매집
- 성장주 일부 차익실현

**중기 (금리 인하 시작)**
- 성장주 재진입
- 중소형주 비중 확대
- 신흥국 주식 고려

**후기 (금리 인하 사이클 중반)**
- 경기 순환주 비중 축소
- 방어주 비중 확대
- 다음 사이클 대비 준비

### ⚠️ 주의사항
- **False Pivot 위험**: 연준이 다시 긴축으로 돌아설 가능성
- **경기 둔화 가능성**: Pivot이 침체 신호일 수도 있음
- **변동성 관리**: 정책 불확실성으로 단기 등락 심함

### 📊 역사적 수익률
- 2019년 Pivot 이후: S&P 500 +30% (1년)
- 2020년 코로나 Pivot: 비트코인 +300% (1년), 나스닥 +80% (1년)

---

## 🔑 시나리오 전환 시그널

### 주의 깊게 모니터링할 지표

1. **고용 지표**
   - 비농업 고용: 월 20만명 이하면 둔화 신호
   - 실업률: 0.3%p 이상 상승 시 침체 경고

2. **인플레이션**
   - CPI, PCE: 목표치(2%) 근접 여부
   - 핵심 인플레이션 추세 변화

3. **연준 발언**
   - FOMC 성명서 문구 변화
   - 점도표(Dot Plot) 변화
   - 의장 기자회견 어조

4. **금융 시장**
   - 수익률 곡선 기울기 변화
   - 신용 스프레드 확대/축소
   - 주택시장 지표

각 시나리오는 명확한 신호와 함께 나타나므로, 금리 데이터를 주간 단위로 모니터링하며 포트폴리오를 선제적으로 조정하는 것이 중요합니다! 🎯